 Week2: comparing about dataset 2
1. [X] whole eu capacity
2. [X] which coutry 
3. [X] total storage per country (each dataset) 
4. [X] how manys pipes in each country
5. [X] capacity per country , speical cross board
6. [X] write the email about api
7. [X] EMAP capacity (check in document about meaning of  s l m a)
8. direct of the pipe
9. eror 

10. [X] storage all 
pipline capcity,结构,start and end, (ad matrix)

In [1]:
import Visualisierungen
import prettytable
import pandas as pd
import json
import re 
from Utility import check,report_capacity

Loading BokehJS ...

Loading BokehJS ...

In [2]:
Visual1=Visualisierungen.Visual()

Dataset
 ----------------
Status: Waiting input!


Component
 ----------------
Status: Waiting input!



Instruction
---------------
choose dataset
please choose dataset from: 

*********************


 ['EMAP_Raw', 'GIE_Raw', 'IGG', 'INET_Filled', 'INET_Raw', 'LKD_Raw', 'NO_Raw'] 


*********************

Example: use your object.choose_dataset(['EMAP_Raw','GIE_Raw']) 

--------------
choose component
please choose component from:


*********************

 ['BorderPoints', 'Compressors', 'ConnectionPoints', 'Consumers', 'EntryPoints', 'InterConnectionPoints', 'LNGs', 'Nodes', 'PipeLines', 'PipePoints', 'PipeSegments', 'Processes', 'Productions', 'Storages'] 


*********************



Example: use your object.component(['Nodes','Nodes']) 



# get all country code

In [3]:

# check who has double country_code?
pure_dict_dataset=Visual1.dataset

country_codes=[]
# pure_dict_dataset is union of seven dataset
for dataset_key in pure_dict_dataset.keys():
    #dive into single dataset
    dataset=pure_dict_dataset[dataset_key].frame_dict
    for table_key in dataset.keys():
        #dive into single table
        table=dataset[table_key]
        try:
            x=table['country_code'].unique()
            #find out what lead to list country_code
            extending=True
            for i in x:
                if len(i.strip())>2:
                    print(table_key)
                    extending=False
                    break
            if extending:
                country_codes.extend(x)
        except:
            print(table.columns)
# set() help remove Duplicate             
print(set(country_codes))



PipeSegments
PipeSegments
PipeSegments
PipeSegments
PipeSegments
PipeSegments
{'AL', 'SA', 'RS', 'BA ', 'AZ', 'AM', 'EE', 'IT', 'AG', 'MK', 'BY', 'BE', 'DK', 'FR', 'TR', 'HR', 'MD', 'ES', 'CH', 'LV', 'NO', 'MT', 'BG', 'BA', 'HU', 'LT', 'GE', 'AT', 'RO', 'LI', 'RU', 'SK ', 'DE', 'PL ', 'CY', 'UA', 'PL', 'GR', 'LU', 'SI', 'SE', 'NL', 'PT', 'IT ', 'CZ', 'IE', 'GB', 'XX', 'LY', 'SK', 'FI'}


In [4]:
# check country code is unified?
# becasue we already have all conutry_code show up, check if there are same country with 
# different code

'''
Conclusion 
1.country code clean!
2. PipeSegments country code tell us about direction
'''

new_list=[x.strip() for x in list(set(country_codes))]
print(sorted(set(new_list)))




['AG', 'AL', 'AM', 'AT', 'AZ', 'BA', 'BE', 'BG', 'BY', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GE', 'GR', 'HR', 'HU', 'IE', 'IT', 'LI', 'LT', 'LU', 'LV', 'LY', 'MD', 'MK', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'RU', 'SA', 'SE', 'SI', 'SK', 'TR', 'UA', 'XX']


# General information
it is convinced that, only param contains important information, uncertainty and method just Describe the information as their names
![](picture/para.png)

<br>
<br>

![](picture/country.png)

maybe xx also XK(Kosovo)

In [29]:
#find xx
for key in Visual1.dataset.keys():
    for sub_key in Visual1.dataset[key].frame_dict.keys():
        if 'XX' in list(Visual1.dataset[key].frame_dict[sub_key]['country_code'].unique()):
            print(key,sub_key)

EMAP_Raw Nodes
EMAP_Raw Productions
EMAP_Raw Storages
IGG Nodes
INET_Filled Nodes
INET_Raw Nodes
LKD_Raw Nodes


In [30]:
#find xk
for key in Visual1.dataset.keys():
    for sub_key in Visual1.dataset[key].frame_dict.keys():
        if 'XK' in list(Visual1.dataset[key].frame_dict[sub_key]['country_code'].unique()):
            print(key,sub_key)

# EMAP

data exploring
----------

![](picture/1.png)
# we can see that the original dataset which sic gird project used to extract emap_raw do have capacity in number

# 94.6% rows has such Magnitude data
![](picture/2.png)

In [5]:
#check 'exact'
EMAP_Raw=Visual1.dataset['EMAP_Raw'].frame_dict

print(EMAP_Raw['Storages']['exact'].isnull().sum())
print()
print(EMAP_Raw['Storages']['exact'].value_counts())
#no missing value 
#only value :'3' means Accuracy is between 10 and 100km

0

3    238
Name: exact, dtype: int64


* exact useless, capacity only in pipesegements
* according to doc Capacity information only show in pipesegements
* inside direction are showed with node_id

capacity
--------

In [6]:
#prepare data
EMAP_Capacity_data=EMAP_Raw['PipeSegments']
EMAP_Capacity_data=EMAP_Capacity_data[['pipe_class_EMap','country_code']]

onehotencoding=pd.get_dummies(EMAP_Capacity_data['pipe_class_EMap'].fillna('No_info'))
onehotencoding['country_code']=EMAP_Capacity_data.country_code
onehotencoding['total_pipeline_numbers']=1

# solve ' problem, this cause json loads error
p = re.compile('(?<!\\\\)\'')

onehotencoding.country_code=onehotencoding.country_code.apply(lambda x:p.sub('\"', x) )

onehotencoding.country_code=onehotencoding.country_code.apply(lambda x : json.loads(x))

onehotencoding['start']=onehotencoding.country_code.apply(lambda x: x[0])
onehotencoding['end']=onehotencoding.country_code.apply(lambda x: x[1])

In [7]:
# whole eu
print('Whole Europa')
onehotencoding[['S','M','L','A','No_info','total_pipeline_numbers']].sum()

Whole Europa


S                         2001
M                         1183
L                         1203
A                          477
No_info                    282
total_pipeline_numbers    5146
dtype: int64

In [31]:
# only inside country
print('inside country')
inside=onehotencoding[onehotencoding.start==onehotencoding.end]
inside=inside.groupby(by='start').sum()[['S','M','L','A','No_info','total_pipeline_numbers']]
inside

inside country


,S,M,L,A,No_info,total_pipeline_numbers
start,,,,,,
AM,3.0,6.0,1.0,0.0,0.0,10
AT,14.0,1.0,4.0,0.0,3.0,22
AZ,3.0,1.0,8.0,0.0,0.0,12
BA,1.0,0.0,0.0,0.0,0.0,1
BE,9.0,7.0,4.0,0.0,1.0,21
BG,37.0,23.0,11.0,0.0,14.0,85
BY,39.0,10.0,24.0,0.0,15.0,88
CH,2.0,0.0,0.0,0.0,1.0,3
CY,0.0,0.0,0.0,2.0,0.0,2


In [32]:
# only cross board
print('cross_board')
crossboard=onehotencoding[onehotencoding.start!=onehotencoding.end]
crossboard=crossboard.groupby(by=['start','end']).sum()[['S','M','L','A','No_info','total_pipeline_numbers']]
crossboard

cross_board


S  M  L  A  No_info  total_pipeline_numbers
start end                                             
AM    AZ   0  1  0  0        0                       1
      TR   0  1  0  0        0                       1
AT    DE   3  0  1  0        0                       4
      HR   3  0  0  0        0                       3
      LI   0  0  0  1        0                       1
...       .. .. .. ..      ...                     ...
XX    NL   3  0  6  0        0                       9
      NO   0  0  6  1        0                       7
      RU   0  0  3  0        0                       3
      TR   0  1  0  0        0                       1
      UA   1  1  1  0        0                       3

[115 rows x 6 columns]

storage
---------

In [10]:
#only how many storage point
EMAP_Storage_data=EMAP_Raw['Storages']

In [11]:
EMAP_Storage_data['country_code'].value_counts()

RU    61
DE    42
FR    14
TR    13
UA    11
BY    10
PL     9
IT     9
GB     8
NL     7
RO     7
CZ     7
BE     6
SK     5
AT     5
HU     5
AM     4
ES     3
DK     2
XX     2
PT     1
GR     1
SE     1
HR     1
LV     1
RS     1
BG     1
AL     1
Name: country_code, dtype: int64

# GIE


In [12]:
GIE_Raw=Visual1.dataset['GIE_Raw'].frame_dict

![](picture/GIE_LNG.png)

In [13]:
print('whole eu')
print('-----------------')
print(GIE_Raw['LNGs'][['max_cap_store2pipe_M_m3_per_d', 'max_workingGas_M_m3',
       'median_cap_store2pipe_M_m3_per_d']].sum())

print('\n#############\n#############\n')
print('country level')
print('-----------------')
GIE_Raw['LNGs'][['country_code','max_cap_store2pipe_M_m3_per_d', 'max_workingGas_M_m3',
       'median_cap_store2pipe_M_m3_per_d']].groupby(by='country_code').sum()

whole eu
-----------------
max_cap_store2pipe_M_m3_per_d        628.820590
max_workingGas_M_m3                 5146.780320
median_cap_store2pipe_M_m3_per_d     618.836617
dtype: float64

#############
#############

country level
-----------------


,max_cap_store2pipe_M_m3_per_d,max_workingGas_M_m3,median_cap_store2pipe_M_m3_per_d
country_code,,,
BE,41.906288,223.08800,41.906288
ES,182.577543,1932.26328,180.135759
FR,118.874328,800.08000,118.101254
GB,121.994909,1038.39872,121.994909
GR,25.445461,75.92000,19.373998
IT,48.524559,249.46144,48.505704
LT,11.831809,97.56888,11.831809
NL,43.461865,315.36000,43.461865
PL,15.348355,186.88000,15.348355


![](picture/GIE_Storages.png)

In [14]:
print('whole eu')
print('-----------------')
print(GIE_Raw['Storages'][['max_cap_pipe2store_M_m3_per_d',
       'max_cap_store2pipe_M_m3_per_d', 'max_workingGas_M_m3']].sum())

print('\n#############\n#############\n')
print('country level')
print('-----------------')
GIE_Raw['Storages'][['country_code','max_cap_pipe2store_M_m3_per_d',
       'max_cap_store2pipe_M_m3_per_d', 'max_workingGas_M_m3']].groupby(by='country_code').sum()

whole eu
-----------------
max_cap_pipe2store_M_m3_per_d    2208.998774
max_cap_store2pipe_M_m3_per_d    1412.418214
max_workingGas_M_m3              1444.535900
dtype: float64

#############
#############

country level
-----------------


,max_cap_pipe2store_M_m3_per_d,max_cap_store2pipe_M_m3_per_d,max_workingGas_M_m3
country_code,,,
AT,98.996889,78.489677,92.2185
BE,15.980013,8.309607,9.0013
BG,3.412841,3.214858,6.2700
CZ,48.823418,36.753088,34.8316
DE,668.718771,376.088432,254.1071
DK,18.553785,9.503158,10.8200
ES,18.900726,11.964740,31.6188
FR,228.405770,114.681814,135.0089
GB,85.823513,71.776186,9.2462


![](picture/GIE_Nodes.png)
there isn't info about capacity, storage

# IGG
![](picture/way2createIGG.png)

when merge, no more information about unit, only about the number of reccords in different dataset and the final dataset, so I suppose that they use the same units

storage LNGs like GIE

GSE only have storage


IGG=INET_filled+INET_raw(Components that not covered by INET_filled)+GIE+GSE

In [18]:
report_capacity(Visual1.dataset['IGG'].frame_dict)

+-----------------------+-------+
|       Component       | Count |
+-----------------------+-------+
|      BorderPoints     |  118  |
|      Compressors      |  230  |
|    ConnectionPoints   |  117  |
|      EntryPoints      |   37  |
| InterConnectionPoints |  117  |
|          LNGs         |   29  |
|         Nodes         |  667  |
|      PipeSegments     |  920  |
|        Storages       |  216  |
+-----------------------+-------+
BorderPoints
------------------------------
-------------------------------


Compressors
------------------------------
eu
 max_cap_M_m3_per_d    8422.499237
dtype: float64
+++++++++++++++++++++++++++++++++
country               max_cap_M_m3_per_d
country_code                    
AT                    280.791392
BE                    140.395696
BG                    280.791392
BY                    210.593544
CH                     35.098924
CZ                    210.593544
DE                   1239.264550
DK                     35.098924
EE          

# other dataset

In [20]:
report_capacity(Visual1.dataset['INET_Filled'].frame_dict)

+--------------+-------+
|  Component   | Count |
+--------------+-------+
| Compressors  |   13  |
|    Nodes     |  938  |
| PipeSegments |  1261 |
| Productions  |   6   |
|   Storages   |   14  |
+--------------+-------+
Compressors
------------------------------
-------------------------------


Nodes
------------------------------
-------------------------------


PipeSegments
------------------------------
eu
 max_cap_M_m3_per_d    1.088431e+07
dtype: float64
+++++++++++++++++++++++++++++++++
inside country
        max_cap_M_m3_per_d
start                    
AT           0.000000e+00
CH           0.000000e+00
DE           1.033124e+07
NL           0.000000e+00
PL           0.000000e+00
XX           0.000000e+00
+++++++++++++++++++++++++++++++++
crossboard
            max_cap_M_m3_per_d  pipenumber
start end                                
AT    DE         40246.818137           2
BE    BG             0.000000           1
BG    BE             0.000000           1
DE    AT       

In [24]:
report_capacity(Visual1.dataset['INET_Raw'].frame_dict)

+-----------------------+-------+
|       Component       | Count |
+-----------------------+-------+
|      BorderPoints     |  119  |
|      Compressors      |  249  |
|    ConnectionPoints   |  118  |
|      EntryPoints      |   37  |
| InterConnectionPoints |  118  |
|          LNGs         |   32  |
|         Nodes         |  908  |
|      PipeSegments     |  920  |
|        Storages       |  199  |
+-----------------------+-------+
BorderPoints
------------------------------
-------------------------------


Compressors
------------------------------
eu
 max_cap_M_m3_per_d    1059.552
dtype: float64
+++++++++++++++++++++++++++++++++
country               max_cap_M_m3_per_d
country_code                    
AG                         0.000
AT                         0.000
BE                         0.000
BG                         0.000
BY                         0.000
CH                         0.000
CZ                         0.000
DE                       467.760
DK             

In [25]:
report_capacity(Visual1.dataset['NO_Raw'].frame_dict)

+--------------+-------+
|  Component   | Count |
+--------------+-------+
|    Nodes     |   57  |
| PipeSegments |   47  |
+--------------+-------+
Nodes
------------------------------
-------------------------------


PipeSegments
------------------------------
eu
 max_cap_M_m3_per_d    739.0
dtype: float64
+++++++++++++++++++++++++++++++++
inside country
        max_cap_M_m3_per_d
start                    
NO                  573.0
+++++++++++++++++++++++++++++++++
crossboard
            max_cap_M_m3_per_d  pipenumber
start end                                
GB    NO                 39.0           1
NO    BE                  0.0           1
      DE                  0.0           3
      FR                 55.0           1
      GB                 72.0           1
-------------------------------




In [23]:
report_capacity(Visual1.dataset['LKD_Raw'].frame_dict)

+-----------+-------+
| Component | Count |
+-----------+-------+
|    LNGs   |   21  |
|   Nodes   |  115  |
|  Storages |  109  |
+-----------+-------+
LNGs
------------------------------
eu
 max_cap_store2pipe_M_m3_per_d       628.820590
median_cap_store2pipe_M_m3_per_d    618.836617
dtype: float64
+++++++++++++++++++++++++++++++++
country               max_cap_store2pipe_M_m3_per_d  median_cap_store2pipe_M_m3_per_d
country_code                                                                 
BE                                41.906288                         41.906288
ES                               182.577543                        180.135759
FR                               118.874328                        118.101254
GB                               121.994909                        121.994909
GR                                25.445461                         19.373998
IT                                48.524559                         48.505704
LT                             

In [26]:
IGG=Visual1.dataset['IGG'].frame_dict
check(IGG)
#check which resource INET IGG use

 

P=IGG['PipeSegments']
print(P['max_cap_M_m3_per_d'].value_counts())
print(Visual1.dataset['INET_Raw'].frame_dict['PipeSegments']['max_cap_M_m3_per_d'].value_counts())
#answer:after filling

+-----------------------+-------+
|       Component       | Count |
+-----------------------+-------+
|      BorderPoints     |  118  |
|      Compressors      |  230  |
|    ConnectionPoints   |  117  |
|      EntryPoints      |   37  |
| InterConnectionPoints |  117  |
|          LNGs         |   29  |
|         Nodes         |  667  |
|      PipeSegments     |  920  |
|        Storages       |  216  |
+-----------------------+-------+
43.143146    416
61.196380     91
59.838202     55
39.012654     22
60.273973     18
            ... 
99.251352      1
63.995552      1
12.050000      1
66.126864      1
43.830000      1
Name: max_cap_M_m3_per_d, Length: 112, dtype: int64
None           775
60.2739726      18
38.08219        12
90.4109589      11
54.79452055     10
35.06849         8
150.0            8
13.6986          8
80.0             7
42.4657          6
6.36             4
8.33             4
8.767123         4
76.71232877      4
100.0            4
58.63013699      3
27.39726       